<a href="https://colab.research.google.com/github/harjeet88/google_collab/blob/main/spark_exmples/day10_leetcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!chmod -R +x /content/drive/MyDrive/installs/

In [3]:
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']
# Verify installation
!java -version
!which java

openjdk version "11.0.2" 2019-01-15
OpenJDK Runtime Environment 18.9 (build 11.0.2+9)
OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
/content/drive/MyDrive/installs/jdk-11.0.2//bin/java


In [4]:
import pyspark
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ["SPARK_HOME"] = "drive/MyDrive/installs/spark-3.5.1-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/installs/py_package') # Replace with your actual path

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
MAX_MEMORY="8g"
maven_coords = [
    "org.apache.spark:spark-avro_2.12:3.2.1",
    "io.delta:delta-core_2.12:2.0.0rc1",
    "org.xerial:sqlite-jdbc:3.36.0.3",
    "graphframes:graphframes:0.8.2-spark3.2-s_2.12",
    "com.acervera.osm4scala:osm4scala-spark3-shaded_2.12:1.0.8",
]

spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.jars.packages", ",".join(maven_coords))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .enableHiveSupport()
    .getOrCreate()
    )

In [8]:
spark

In [9]:
from pyspark.sql.functions import *
from os.path import join
from ast import alias

#User Activity for the Past 30 Days I - 1141
https://leetcode.com/problems/user-activity-for-the-past-30-days-i/description/?envType=study-plan-v2&envId=top-sql-50

In [10]:
%%writefile /content/drive/MyDrive/data/leetcode_sql180/user_activity_1141.csv
user_id,session_id,activity_date,activity_type
1,1,2019-07-20,open_session
1,1,2019-07-20,scroll_down
1,1,2019-07-20,end_session
2,4,2019-07-20,open_session
2,4,2019-07-21,send_message
2,4,2019-07-21,end_session
3,2,2019-07-21,open_session
3,2,2019-07-21,send_message
3,2,2019-07-21,end_session
4,3,2019-06-25,open_session
4,3,2019-06-25,end_session

Overwriting /content/drive/MyDrive/data/leetcode_sql180/user_activity_1141.csv


In [14]:
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- session_id: integer (nullable = true)
 |-- activity_date: date (nullable = true)
 |-- activity_type: string (nullable = true)



In [11]:
df= spark.read.csv("/content/drive/MyDrive/data/leetcode_sql180/user_activity_1141.csv",inferSchema=True,header=True)
df.show()

+-------+----------+-------------+-------------+
|user_id|session_id|activity_date|activity_type|
+-------+----------+-------------+-------------+
|      1|         1|   2019-07-20| open_session|
|      1|         1|   2019-07-20|  scroll_down|
|      1|         1|   2019-07-20|  end_session|
|      2|         4|   2019-07-20| open_session|
|      2|         4|   2019-07-21| send_message|
|      2|         4|   2019-07-21|  end_session|
|      3|         2|   2019-07-21| open_session|
|      3|         2|   2019-07-21| send_message|
|      3|         2|   2019-07-21|  end_session|
|      4|         3|   2019-06-25| open_session|
|      4|         3|   2019-06-25|  end_session|
+-------+----------+-------------+-------------+



In [12]:
df1 = df.groupBy("activity_date").agg(countDistinct("user_id").alias("user_count"))
df1.show()

+-------------+----------+
|activity_date|user_count|
+-------------+----------+
|   2019-07-21|         2|
|   2019-06-25|         1|
|   2019-07-20|         2|
+-------------+----------+



In [13]:
#.filter(col("activity_date")>"2019-06-27" & col("activity_date")<="2019-07-27")
df1.where(df.activity_date.between("2019-06-28","2019-07-27")).show()

+-------------+----------+
|activity_date|user_count|
+-------------+----------+
|   2019-07-21|         2|
|   2019-07-20|         2|
+-------------+----------+



In [15]:
df2 = df.where(df.activity_date.between("2019-06-28","2019-07-27"))
df.show()


+-------+----------+-------------+-------------+
|user_id|session_id|activity_date|activity_type|
+-------+----------+-------------+-------------+
|      1|         1|   2019-07-20| open_session|
|      1|         1|   2019-07-20|  scroll_down|
|      1|         1|   2019-07-20|  end_session|
|      2|         4|   2019-07-20| open_session|
|      2|         4|   2019-07-21| send_message|
|      2|         4|   2019-07-21|  end_session|
|      3|         2|   2019-07-21| open_session|
|      3|         2|   2019-07-21| send_message|
|      3|         2|   2019-07-21|  end_session|
|      4|         3|   2019-06-25| open_session|
|      4|         3|   2019-06-25|  end_session|
+-------+----------+-------------+-------------+



In [16]:
df2.groupBy("activity_date").agg(countDistinct("user_id").alias("user_count")).show()

+-------------+----------+
|activity_date|user_count|
+-------------+----------+
|   2019-07-21|         2|
|   2019-07-20|         2|
+-------------+----------+

